In [ ]:
import pandas as pd 

In [ ]:
data = pd.read_csv('tesla_tweets.csv', sep='|')

In [ ]:
data.head()

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
model = BertModel.from_pretrained("bert-base-uncased")

In [ ]:
sample = "We are going to implement sentiment analysis with BERT!"

In [ ]:
tokens = tokenizer.tokenize(sample)
print(tokens)

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

In [ ]:
tokenizer.pad_token, tokenizer.pad_token_id

In [ ]:
tokenizer.unk_token, tokenizer.unk_token_id

In [ ]:
tokenizer.cls_token, tokenizer.cls_token_id

In [ ]:
tokenizer.sep_token, tokenizer.sep_token_id

Tokenizer.encode_plus adds [CLS] at beginning and [SEP] at end of the sentence 

In [ ]:
manual_sample = '[CLS]' + sample + '[SEP]'
tokens = tokenizer.tokenize(manual_sample)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(manual_sample)
print(tokens)
print(token_ids)

In [ ]:
encoding = tokenizer.encode_plus(sample, max_length=24, truncation=True, pad_to_max_length=True,
                                add_special_tokens=True, return_attention_mask=True,
                                return_tensors='tf')

In [ ]:
encoding 

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow as tf
from transformers import DistilBertTokenizerFast

In [ ]:
df = pd.read_csv('train.csv', sep='\t')
df.head()

In [ ]:
df.drop(['PhraseId', 'SentenceId'], axis=1, inplace=True)
df.head()

In [ ]:
df.tail(20)

In [ ]:
df['Sentiment'] = df['Sentiment'].apply(lambda x: 2 if x > 2 else (1 if x == 2 else 0))
df.tail(20)

In [ ]:
seqlen = df['Phrase'].apply(lambda x: len(x.split()))

In [ ]:
seqlen

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(16, 10))
sns.distplot(seqlen)

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
SEQLEN = 30

In [ ]:
def encoder(sentence):
    tokens = tokenizer.encode_plus(
        sentence, max_length=SEQLEN, truncation=True,
        padding='max_length', add_special_tokens=True,
        return_attention_mask=True, return_token_type_ids=False
    )
    return tokens['input_ids'], tokens['attention_mask']

In [ ]:
Xraw = df['Phrase'].values
Xraw[:5]

In [ ]:
input_ids = []
attention_mask = []

for item in Xraw:
    input_i, mask = encoder(item)
    input_ids.append(input_id)
    attention_mask.append(mask)

In [ ]:
Xids = np.asarray(input_ids)
Xmask = np.asarray(attention_mask)

In [ ]:
Xids.shape

In [ ]:
Xmask.shape

In [ ]:
Xmask

In [ ]:
arr = df['Sentiment'].values

In [ ]:
labels = np.zeros((arr.size, arr.max()+1))

In [ ]:
labels[np.arange(arr.size), arr] = 1

In [ ]:
labels 

In [ ]:
with open('xids.npy', 'wb') as f:
    np.save(f, Xids)
    
with open('xmask.npy', 'wb') as f:
    np.save(f, Xmask)
    
with open('labels.npy', 'wb') as f:
    np.save(f, labels)

In [ ]:
import numpy as np
import tensorflow as tf
from transformers import TFDistilBertModel, DistilBertConfig

In [ ]:
tf.config.experimental.list_physical_devices('GPU')

In [ ]:
with open('xids.npy', 'rb') as f:
    Xids = np.load(f)

In [ ]:
Xids

In [ ]:
with open('xmask.npy', 'rb') as f:
    Xmask = np.load(f)
with open('labels.npy', 'rb') as f:
    labels = np.load(f)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [ ]:
for item in dataset.take(1):
    print(item)

In [ ]:
def map_func(input_ids, mask, label):
    return {'input_ids': input_ids, 'attention_mask': mask}, label

In [ ]:
dataset = dataset.map(map_func)

In [ ]:
for item in dataset.take(1):
    print(item)

In [ ]:
dataset = dataset.shuffle(10000).batch(64)

In [ ]:
size = len(dataset)
size

In [ ]:
split = 0.9
train = dataset.take(round(size*split))
val = dataset.skip(round(size*split))

In [ ]:
split*size

In [ ]:
from transformers import TFDistilBertForSequenceClassification
config = DistilBertConfig(num_labels=3)

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', config=config)
model.summary()

In [ ]:
model.layers[0].trainable = False  # Setting distilber layer to freeze
model.summary()

In [ ]:
optimizer = tf.keras.optimizer.Adam(0.02)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [ ]:
history = model.fit(
    train,
    validation_data = val,
    epochs = 20
)

## Building NN models 

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
with open('xids.npy', 'rb') as fp:
    Xids = np.load(fp)
with open('xmask.npy', 'rb') as fp:
    Xmask = np.load(fp)
with open('labels.npy', 'rb') as fp:
    labels = np.load(fp)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [ ]:
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask':masks}, labels

In [ ]:
dataset = dataset.map(map_func)

In [ ]:
BATCH_SIZE = 32
dataset = dataset.shuffle(10000).batch(BATCH_SIZE)

In [ ]:
SPLIT = 0.8
DS_LEN = len(list(dateset))
train = dataset.take(round(DS_LEN*SPLIT))
val = dataset.skip(round(DS_LEN*SPLIT))

In [ ]:
from transformers import TFAutoModel

In [ ]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

In [ ]:
input_ids = tf.keras.layers.Input(shape=(50,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(50,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[0]

X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
X = tf.keras.layers.Dense(32, activation='relu')(X)
y = tf.keras.layers.Dense(3, activate='softmax', name='outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

In [ ]:
model.layers[2].trainable = False  # This time our BERT is 2nd layer

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer, loss=loss, metrics=[acc])

In [ ]:
history = model.fit(
    train,
    validation_data=val,
    epochs=140
)

In [ ]:
model.save('../models/nn140')

In [ ]:
sns.set_style('darkgrid')
mpl.rcParams.update({'font.size': 18})

epochs = list(range(len(history.history['accuracy'])))

plt.figure(figsize=(16, 10))
sns.lineplot(x=epochs, y=history.history['accuracy'], label='acc', color='#08c6ab')
sns.lineplot(x=epochs, y=history.history['val_accuracy'], label='val-acc', color='#212b38')

sns.lineplot(x=epochs, y=history.history['loss'], label='loss', color='#726eff')
sns.lineplot(x=epochs, y=history.history['val_loss'], label='val-loss', color='#37465b')
plt.tight_layout()
plt.savefig('../figures/nn140-metrics.png')

## Bulding ConveNet Models 

In [ ]:
input_ids = tf.keras.layers.Input(shape=(50,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(50,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[0]

X = tf.keras.layers.Conv1D(filters=50, kernel_size=2, padding='valid', activation = 'relu')(embeddings)
X = tf.keras.layers.Conv1D(filters=50, kernel_size=3, padding='valid', activation = 'relu')(X)
X = tf.keras.layers.Conv1D(filters=50, kernel_size=4, padding='valid', activation = 'relu')(X)

X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
X = tf.keras.layers.Dense(32, activation='relu')(X)
y = tf.keras.layers.Dense(3, activate='softmax', name='outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

## Bulding an LSTM model

In [ ]:
input_ids = tf.keras.layers.Input(shape=(50,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(50,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[0]

X = tf.keras.layers.LSTM(64)(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
X = tf.keras.layers.Dense(32, activation='relu')(X)
y = tf.keras.layers.Dense(3, activate='softmax', name='outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

## Final Model Walkthrough

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [ ]:
api = KaggleApi()
api.authenticate()

In [ ]:
api.dataset_download_file('kazanova/sentiment140', file_name='training.1600000.processed.noemoticon.csv', path='./')

In [ ]:
import zipfile 

In [ ]:
with zipfile.ZipFile('training.1600000.processed.noemoticon.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
import pandas as pd 

In [ ]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1',
                names=['target', 'id', 'date', 'flag', 'user', 'text'])

In [ ]:
df.head()

In [ ]:
df['target'].unique()

In [ ]:
df = df [['target', 'text']]
df.head()

In [ ]:
df.groupby('target')['text'].count()

In [ ]:
df[df['target'] == 4].head()

In [ ]:
seqlen = df['text'].apply(lambda x: len(x.split()))

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
sns.set_style('darkgrid')
plt.figure(figsize=(16, 10))
sns.distplot(seqlen)

In [ ]:
SEQLEN = 32

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [ ]:
def tokenize(sentence):
    tokens = tokenizer.encode_plus(sentence, max_length=SEQLEN,
                                  truncation=True, padding='max_length',
                                  add_special_tokens=True, return_attention_mask=True,
                                  return_token_type_ids=False, return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

In [ ]:
import numpy as np

In [ ]:
Xids = np.zeros((len(df), SEQLEN))
Xmask = np.zeros((len(df), SEQLEN))

In [ ]:
Xids.shape

In [ ]:
for i, sentence in enumerate(df['text']):
    Xids[i, :], Xmask[i, :] = tokenize(sentence)
    if i % 100000 == 0:
        print(i)

In [ ]:
Xids

In [ ]:
Xmask

In [ ]:
df['target'] = df['target'].apply(lambda x: 1 if x == 4 else 0)

In [ ]:
arr = df['target'].values

In [ ]:
arr

In [ ]:
labels = np.zeros((arr.size, arr.max()+1))

In [ ]:
labels[np.arange(arr.size), arr] = 1

In [ ]:
labels 

In [ ]:
with open('twitter-xids.npy', 'wb') as f:
    np.save(f, Xids)
with open('twitter-xmask.npy', 'wb') as f:
    np.save(f, Xmask)
with open('twitter-labels.npy', 'wb') as f:
    np.save(f, labels)

In [ ]:
del df, Xids, Xmask, labels 

In [ ]:
import tensorflow as tf

In [ ]:
with open('twitter-xids.npy', 'rb') as f:
    Xids = np.load(f)
with open('twitter-xmask.npy', 'rb') as f:
    Xmask = np.load(f)
with open('twitter-labels.npy', 'rb') as f:
    labels = np.load(f)

In [ ]:
tf.config.experimental.list_physical_devices('GPU')

In [ ]:
data = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [ ]:
SHUFFLE = 10000000
BATCH_SIZE = 32

In [ ]:
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask':masks}, labels

In [ ]:
data = data.map(map_func)

In [ ]:
data = data.shuffle(SHUFFLE).batch(BATCH_SIZE)

In [ ]:
SPLIT = 0.9
train = data.take(int(50000*SPLIT))
val = data.skip(int(50000*SPLIT))
del data

In [ ]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

In [ ]:
input_ids = tf.keras.layers.Input(shape=(SEQLEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQLEN,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[0]

x = tf.keras.layers.LSTM(64)(embeddings)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(x)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model.layers[2].trainable=False

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [ ]:
history = model.fit(
    train,
    validation_data=val,
    epochs=10
)

In [ ]:
model.save('../models/final')

In [ ]:
sns.set_style('darkgrid')
mpl.rcParams.update({'font.size': 18})

epochs = list(range(len(history.history['accuracy'])))

plt.figure(figsize=(16, 10))
sns.lineplot(x=epochs, y=history.history['accuracy'], label='acc', color='#08c6ab')
sns.lineplot(x=epochs, y=history.history['val_accuracy'], label='val-acc', color='#212b38')

sns.lineplot(x=epochs, y=history.history['loss'], label='loss', color='#726eff')
sns.lineplot(x=epochs, y=history.history['val_loss'], label='val-loss', color='#37465b')
plt.tight_layout()
plt.savefig('../figures/final-acc.png')